## Importazione librerie

In [1]:
import pandas as pd
from datetime import datetime

## Caricamento del dataset originale
Il dataset originale è composto da 4 colonne:   
- user_id: identificatore dell'utente
- date: giorno in cui è stato raccolto il dato aggregato
- data_type: intero che identifica la tipologia di dato raccolto
- data_value: valore associato al dato raccolto

I diversi valori della variabile data_type si riferiscono a dati aggregati raccolti giornalmente dal dispositivo di ogni singolo utente e identificano le tipologie di dato in base al seguente schema:
- 1: numero totale di passi effettuati
- 2: peso (kg)
- 3: BMI (kg/m^2)
- 4: pressione sanguigna sistolica (mmHg)
- 5: velocità dell'onda sfigmica arteriosa (PWV), (m/s)
- 6: PWV healthiness (1: bassa, 2: sano, 3: troppo alta)
- 7: frequenza cardiaca media (bpm)
- 8: frequenza cardiaca minima (bpm)
- 9: frequenza cardiaca massima (bpm)
- 10: durata del sonno (ore)
- 11: orario in cui l'utente si è messo a letto
- 12: orario in cui l'utente si è alzato dal letto
- 13: numero di volte in cui l'utente si è svegliato durante il sonno
- 14: durata del tempo in cui l'utente si è svegliato durante il sonno (ore)
- 15: tempo impiegato dall'utente per addormentarsi (ore)
- 16: tempo impiegato dall'utente per alzarsi dal letto (ore)
- 17: durata di sonno leggero (ore)
- 18: durata di sonno REM (ore)
- 19: durata di sonno profondo (ore)
- 20: tipo di attività
- 21: durata dell'attività (secondi)
- 22: calorie consumate durante l'attività
- 23: frequenza cardiaca media durante l'attività (bpm)
- 24: frequenza cardiaca minima durante l'attività (bpm)
- 25: frequenza cardiaca massima durante l'attività (bpm)
- 26: velocità dell'andatura dei passi (passi al minuto)
- 27: velocità dell'andatura a distanza (km all'ora)

In [2]:
raw_dataset = pd.read_csv('./data.csv', usecols=['user_id', 'date', 'data_type', 'data_value'])

raw_dataset.head()

,user_id,date,data_type,data_value
0,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,1,10131.39
1,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,2,86.80
2,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,3,28.96
3,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,7,117.62
4,2bc16eda651db5936cd31e735c815296fc1579d9,2016-04-01,8,117.86


## Creazione del dataset personalizzato
L'obiettivo del progetto è quello di verificare, attraverso strumenti di deep learning, l'esistenza di una correlazione fra la qualità del sonno di un utente e l'andatura dei suoi passi. Per raggiungere tale scopo è necessario costruire un nuovo dataset personalizzato contenente i soli dati utili per questa analisi, in particolare i dati riguardanti il sonno e l'andatura dei passi degli utenti.

In [19]:
feature_types = [10, 11, 12, 14, 15, 18, 19, 26]
feature_names = ["sleepduration", "bedin", "bedout", "awakeduration", "timetosleep", "remduration", "deepduration", "stepsgaitspeed"]

dataset_array = []

for i in range(len(feature_types)):
    dataset_feature = raw_dataset[raw_dataset['data_type'] == feature_types[i]].copy(deep=True)
    dataset_feature.drop(raw_dataset.columns[[2]], axis=1, inplace=True)
    dataset_feature.columns = ["user_id", "date", feature_names[i]]
    dataset_array.append(dataset_feature)

dataset = dataset_array[0]
for i in range(1, len(dataset_array)):
    dataset = dataset.merge(dataset_array[i], on=["user_id", "date"], how="outer")

# rimozione delle righe del dataset che non contengono i valori più importanti per lo studio
dataset = dataset.dropna(subset=['sleepduration', 'stepsgaitspeed'])

dataset.set_index(['user_id','date'], inplace=True)

# salvataggio del dataset in un file locale
dataset.to_csv('dataset.csv')
dataset.head()

sleepduration  bedin  \
user_id                                  date                               
1003e58667235e01b49008155604980b3900b00e 2016-08-01          11.31  22.11   
                                         2016-08-10           9.15  22.34   
                                         2016-08-15           7.87  23.93   
                                         2016-09-05           8.34  22.70   
                                         2016-09-06           8.08  23.55   

                                                     bedout  awakeduration  \
user_id                                  date                                
1003e58667235e01b49008155604980b3900b00e 2016-08-01    9.86           0.53   
                                         2016-08-10    7.66           0.28   
                                         2016-08-15    7.95           0.27   
                                         2016-09-05    7.16           0.28   
                                         2016-09-06    7.72           0.20   

                                                     timetosleep  remduration  \
user_id                                  date                                   
1003e58667235e01b49008155604980b3900b00e 2016-08-01         0.15          NaN   
                                         2016-08-10         0.02          NaN   
                                         2016-08-15         0.02          NaN   
                                         2016-09-05         0.10          NaN   
                                         2016-09-06         0.00          NaN   

                                                     deepduration  \
user_id                                  date                       
1003e58667235e01b49008155604980b3900b00e 2016-08-01          3.99   
                                         2016-08-10          3.58   
                                         2016-08-15          3.19   
                                         2016-09-05          3.11   
                                         2016-09-06          3.60   

                                                     stepsgaitspeed  
user_id                                  date                        
1003e58667235e01b49008155604980b3900b00e 2016-08-01          108.81  
                                         2016-08-10          105.02  
                                         2016-08-15           88.27  
                                         2016-09-05           84.91  
                                         2016-09-06           88.60

## Analisi del dataset personalizzato


In [2]:
dataset = pd.read_csv('./dataset.csv')

Per estrarre delle sequenze di giorni significative, è necessario trovare nei dati delle sequenze di giorni che non abbiano troppi dati mancanti. Nello studio sono state selezionate delle sequenze di giorni di lunghezza 15 in cui ogni giorno è seguito al più da un singolo giorno mancante.

In [3]:
# metodo che trasforma una data in un intero (la data zero è allineata a quella del dataset e corrisponde al giorno 2016-04-01)
def date_to_int(date):
    d1 = datetime.strptime('2016-04-01', "%Y-%m-%d")
    d2 = datetime.strptime(date, "%Y-%m-%d")
    return abs((d2 - d1).days)

In [4]:
# trasformazione delle date in interi per semplificare il confronto
dataset['date'] = dataset['date'].apply(date_to_int)
dataset.set_index(['user_id','date'], inplace=True)
dataset.head()

sleepduration  bedin  bedout  \
user_id                                  date                                 
1003e58667235e01b49008155604980b3900b00e 122           11.31  22.11    9.86   
                                         131            9.15  22.34    7.66   
                                         136            7.87  23.93    7.95   
                                         157            8.34  22.70    7.16   
                                         158            8.08  23.55    7.72   

                                               awakeduration  timetosleep  \
user_id                                  date                               
1003e58667235e01b49008155604980b3900b00e 122            0.53         0.15   
                                         131            0.28         0.02   
                                         136            0.27         0.02   
                                         157            0.28         0.10   
                                         158            0.20         0.00   

                                               remduration  deepduration  \
user_id                                  date                              
1003e58667235e01b49008155604980b3900b00e 122           NaN          3.99   
                                         131           NaN          3.58   
                                         136           NaN          3.19   
                                         157           NaN          3.11   
                                         158           NaN          3.60   

                                               stepsgaitspeed  
user_id                                  date                  
1003e58667235e01b49008155604980b3900b00e 122           108.81  
                                         131           105.02  
                                         136            88.27  
                                         157            84.91  
                                         158            88.60

In [10]:
target_sequence_len = 15
last_user = None
last_date = None
current_sequence = []
sequences = []
count = 1

# estrazione di sequenze di lunghezza target_sequence_len che soddisfano i criteri stabiliti (ogni giorno è seguito al più da un singolo giorno mancante)
for index, row in dataset.iterrows():
    user, date = index
    if last_user != None:
        if user == last_user:
            if date == last_date + 1 or date == last_date + 2:
                current_sequence.append((index, row))
                if len(current_sequence) == target_sequence_len:
                    sequences.append(current_sequence[::])
                    current_sequence.pop(0)
            else:
                current_sequence = []
        else:
            current_sequence = []
    else:
        current_sequence.append((index, row))
    last_user = user
    last_date = date

print(f'trovate {len(sequences)} sequenze di lunghezza {target_sequence_len}')

trovate 273472 sequenze di lunghezza 15


In [11]:
# visualizzazione dei giorni considerati per alcune sequenze
for sequence in sequences[20:30]:
    test_sequence = []
    for index, row in sequence:
        test_sequence.append(index[1])
    print(test_sequence)

[344, 345, 346, 348, 349, 351, 352, 353, 354, 355, 357, 358, 359, 360, 361]
[345, 346, 348, 349, 351, 352, 353, 354, 355, 357, 358, 359, 360, 361, 362]
[346, 348, 349, 351, 352, 353, 354, 355, 357, 358, 359, 360, 361, 362, 364]
[172, 173, 174, 175, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188]
[173, 174, 175, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189]
[174, 175, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191]
[175, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191, 192]
[176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191, 192, 193]
[178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194]
[179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194, 195]


Per poter estrarre dalle sequenze di giorni dei valori rilevanti più semplici da confrontare, è necessario addestrare un modello sull'analisi delle sequenze. In questo progetto è stato scelto come task di addestramento dei modelli il task di forecasting. Dunque, data una sequenza di 14 giorni contenente i valori di una variabile nei diversi giorni come input, il modello dovrà cercare di prevedere il valore di tale variabile al giorno 15.
Per poter soddisfare gli obiettivi del progetto è necessario addestrare due modelli diversi, uno per la previsione della qualità del sonno e l'altro per la previsione dell'andatura dei passi. È dunque necessario creare i dati di addestramento per tali modelli.

Per quanto riguarda la qualità del sonno, questa viene calcolata attraverso una versione rivisitata del Pittsburgh Sleep Quality Index (PSQI) in modo da poter includere nel calcolo dell'indice anche i dati riguardanti la durata del sonno profondo e la durata del sonno REM. Per addestrare il modello per la previsione della qualità del sonno è quindi necessario trasformare 